In [9]:
import torch.nn as nn
import torch.nn.functional as F # 파라미터가 없는 함수들 모임
import torch

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
# 오류가 났던 모델
class Net(nn.Module):
    def __init__(self):
        super().__init__() # 32*32
        self.conv1 = nn.Conv2d(1, 6, 5) #32-(5)+1 =28
        self.pool1 = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1= nn.Linear(400, out_features=120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))  # 6, 14, 14
        x = self.pool2(F.relu(self.conv2(x))) #16, 5, 5
        x = torch.flatten(x,1)  # 400
        x = self.fc2(F.relu(self.fc1(x))) # 84
        x = self.fc3(F.relu(x)) # 10
        return x

In [6]:
# 수정한 모델
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1= nn.Linear(400, out_features=120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))  # 6, 14, 14
        x = self.pool2(F.relu(self.conv2(x))) #16, 5, 5
        x = torch.flatten(x,0)  # 400
        x = self.fc2(F.relu(self.fc1(x))) # 84
        x = self.fc3(F.relu(x)) # 10
        return x

https://pytorch.org/docs/stable/generated/torch.flatten.html
> flatten을 쓰는 법!
> dim = 0은 가장 밖. 그것부터 flatten할 것인지에 대한 뜻.

결과를 이렇게 하나 하나 확인해볼 수 있다!

In [12]:
input_image = torch.rand(3, 1, 32, 32)
conv1 = nn.Conv2d(1, 6, 5)
pool1 = nn.MaxPool2d(2,2)
x = conv1(input_image)
print("after conv1", x[0,0,0:2])
x = pool1(x)
print("after pool1", x[0,0,0:2])


after conv1 tensor([[ 0.1767,  0.5099,  0.2811,  0.1192,  0.2436,  0.2181,  0.2618,  0.3569,
          0.1170,  0.4378,  0.1879,  0.2645,  0.2526,  0.1127,  0.4620,  0.3147,
          0.2179, -0.1752,  0.4312,  0.2890,  0.2204,  0.1696,  0.2275,  0.5134,
          0.1934,  0.2623, -0.0919,  0.4461],
        [ 0.1105,  0.2790,  0.2349,  0.3074,  0.0452,  0.4467,  0.2779,  0.2666,
          0.3471,  0.1146,  0.3940,  0.1031,  0.5431, -0.0803,  0.1840,  0.3206,
          0.4485,  0.0346,  0.1839,  0.0809,  0.3556,  0.2699,  0.1056,  0.2724,
          0.1099,  0.2503,  0.1501,  0.2629]], grad_fn=<SliceBackward0>)
after pool1 tensor([[0.5099, 0.3074, 0.4467, 0.3569, 0.4378, 0.3940, 0.5431, 0.4620, 0.4485,
         0.4312, 0.3556, 0.5134, 0.2623, 0.4461],
        [0.2270, 0.4986, 0.4605, 0.3856, 0.5990, 0.5626, 0.4731, 0.4172, 0.4538,
         0.3046, 0.4045, 0.7254, 0.6088, 0.3371]], grad_fn=<SliceBackward0>)


In [7]:
model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
# 데이터입력값
X = torch.rand(1, 32, 32, device = device) # torch.rand(3,1,32,32) 첫번째는 배치 사이즈
# 원시 예측값 (2차원 텐서 반환. dim=0: 각 class에 대한 raw_pred 10개, dim=1: 각 출력의 개별 값)
logits = model(X)
logits

tensor([ 0.0967, -0.0296, -0.1176, -0.0413,  0.0529,  0.0548, -0.0302,  0.0478,
        -0.0653, -0.1082], device='cuda:0', grad_fn=<ViewBackward0>)